In [ ]:
import pandas as pd
import numpy as np
import time
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
colNames = ['hoeveel', 'wanneer', 'laat', 'lang', 'kwaliteit']
data = pd.read_csv(filepath_or_buffer=r'C:\Users\Ricardo\OneDrive - Avans Hogeschool\1 school HBO\School HBO v2\4.1\Human Centered AI Design\web application\Cafeïne en slaaptekort.csv'
                    , sep=";", header=0, names=colNames, usecols=[1,2,3,4,5], encoding = "ISO-8859-1")

data

In [ ]:
def categorise_hoeveel(hoeveel):
    if hoeveel == "Geen":
        return 0
    elif hoeveel == "1 tot 2 per dag":
        return 1
    elif hoeveel == "3 tot 4 per dag":
        return 3
    elif hoeveel == "5 tot 6 per dag":
        return 5
    elif hoeveel == "meer dan 6 per dag":
        return 7

def categorise_wanneer(wanneer):
    if wanneer == "Voor of tijdens de lunch":
        return 0
    elif wanneer == "Tussen 15:00 en 17:00":
        return 15
    elif wanneer == "Tussen 17:00 en 19:00":
        return 17
    elif wanneer == "Tussen 19:00 en 21:00":
        return 19
    elif wanneer == "Tussen 21:00 en 23:00":
        return 21
    elif wanneer == "Na 23:00":
        return 23

def categorise_laat(laat):
    if laat == "Tussen 21:00 en 22:00":
        return 21
    elif laat == "Tussen 22:00 en 23:00":
        return 22
    elif laat == "Tussen 23:00 en 00:00":
        return 23
    elif laat == "Tussen 00:00 en 02:00":
        return 24
    elif laat == "Tussen 02:00 en 04:00":
        return 26

def categorise_lang(lang):
    if lang == "Minder dan 10 min":
        return 0
    elif lang == "Tussen 10 min en 30 min":
        return 1
    elif lang == "Tussen 30 min en 1 uur":
        return 2
    elif lang == "Meer dan 1 uur":
        return 3

dataHeat = data.copy(deep=True)

dataHeat['hoeveel'] = data['hoeveel'].apply(categorise_hoeveel)
dataHeat['wanneer'] = data['wanneer'].apply(categorise_wanneer)
dataHeat['laat'] = data['laat'].apply(categorise_laat)
dataHeat['lang'] = data['lang'].apply(categorise_lang)


dataHeat

In [ ]:
wanneerCounts = dataHeat['wanneer'].value_counts()
wanneerCounts

In [ ]:
corData = dataHeat[['wanneer', 'lang']]
corData.corr()

In [ ]:
sns.heatmap(dataHeat.corr(), annot = True, vmin=-1, vmax=1, center= 0)

In [ ]:
from tensorflow.keras.utils import to_categorical
# X = dataHeat.copy(deep=True)
# del X['lang'], X['kwaliteit']
#
# Y = dataHeat.copy(deep=True)
# del Y['hoeveel'], Y['wanneer'], Y['laat'], Y['kwaliteit']
# Y
X = dataHeat.drop(["hoeveel", "lang", "kwaliteit"],  axis=1)
#X.drop('kwaliteit', axis=1)
Xlist = X.values.tolist()

Y = dataHeat.drop(["hoeveel", "wanneer", "laat", "kwaliteit"], axis=1)
Ycat = to_categorical(Y)
Ylist = Ycat.tolist()
Xlist

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, Adagrad, RMSprop
#from tensorflow.keras.utils import to_categorical
#from sklearn.model_selection import train_test_split

In [ ]:
model = Sequential()
epochs = 500

model.add(Dense(2, input_shape=(2,), activation='relu'))
model.add(Dense(3, activation='tanh'))
#model.add(Dense(4, activation='relu'))
model.add(Dense(5, activation='tanh'))
model.add(Dense(4, activation='softmax'))
model.compile(Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy']) #binary_crossentropy

history = model.fit(Xlist, Ylist, epochs=epochs, batch_size=50, verbose=2) #batch_size=10,


In [ ]:
loss_train = history.history['loss']
acc_train = history.history['accuracy']
epochs = range(0,epochs)

# plt.subplot(1,2,1)
# plt.plot(epochs, loss_train, 'g', label='Training loss')
# plt.title('Training loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.subplot(1,2,2)
# plt.plot(epochs, acc_train, 'g', label='Training accuracy')
# plt.title('Training and Validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.show()

fig = plt.figure()
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

fig = plt.figure()
plt.plot(epochs, acc_train, 'g', label='Training accuracy')
plt.title('Training  accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
# Save the weights
#model.save('model_save/model84')

# Restore the weights
#model.load_weights('./checkpoints/my_checkpoint')

new_model = tf.keras.models.load_model('model_save/model84')

# Evaluate the model
loss, acc = new_model.evaluate(Xlist, Ylist, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))